Zusammenhang zwischen PM25-Werten und Wetter

erst mal NUR in Atlanta, dann in Hamburg



In [ ]:
# imports
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

In [ ]:
df.columns

In [ ]:
# Filtern des DataFrames für Hamburg und Atlanta
df_atlanta = df[df['City'].isin(['Atlanta'])]

# Überprüfung der ersten Zeilen des gefilterten DataFrames
print(df_atlanta.head())

Df erstellen mit Spalten für Zeit, Zielschadstoff (PM2.5) und ausgewählten Wettereigenschaften, die 

In [ ]:
# Auswahl der relevanten Spalten für die Analyse
df_atlanta_pm25 = df_atlanta[['Year', 'Month', 'Day', 'Pm25', 'Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']]

# Überprüfung der ersten Zeilen des neuen DataFrames
print(df_atlanta_pm25.head())


In [ ]:
# Berechnung der Korrelationsmatrix
correlation_matrix = df_atlanta_pm25[['Pm25', 'Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']].corr()

# Visualisierung der Korrelationsmatrix mit 'center=0'
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', center=0)
plt.title('Korrelation zwischen PM2.5 und Wetterdaten in Atlanta')
plt.show()


Die negative Korrelation mit der Windgeschwindigkeit ist die stärkste. Das bedeutet, dass in Zeiten mit wenig Wind die PM2.5-Werte tendenziell höher sind, was darauf hindeutet, dass Wind eine wichtige Rolle bei der Verteilung und Dispersion von Feinstaub spielt. Dies ist zu erwarten, da Wind dazu beiträgt, Luftschadstoffe zu vertreiben, während ruhiges Wetter (wenig Wind) zu einer Ansammlung von Schadstoffen führen kann.

In [ ]:
# Erstelle Scatterplots für jede Wettervariable im Vergleich zu PM2.5
variables = ['Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']
plt.figure(figsize=(12, 10))

for i, var in enumerate(variables, 1):
    plt.subplot(2, 3, i)  # 2 Zeilen, 3 Spalten
    plt.scatter(df_atlanta_pm25[var], df_atlanta_pm25['Pm25'], alpha=0.6)
    plt.title(f'Scatterplot: PM2.5 vs {var}')
    plt.xlabel(var)
    plt.ylabel('PM2.5 (µg/m³)')
    plt.grid(True)

plt.tight_layout();


Sagt mir jetzt nicht so viel.

Wir machen jetzt mal eine multiple lineare Regression (also mit mehreren Variablen) und schauen, was da rauskommt.

Aber dazu müssen wir erst die NaN-Werte behandeln.

Möglichkeiten:
Wir haben Zeitreihen, also könnten wir vom Vortag aus imputieren.
Wir können aber auch Mittelwerte nehmen. Und das halte ich hier für sinnvoll.

In [ ]:
# Inputiere NaN-Werte mit dem Median der jeweiligen Spalte
df_atlanta_pm25.fillna(df_atlanta_pm25.median(), inplace=True)

# Überprüfung, ob NaN-Werte noch vorhanden sind
print(df_atlanta_pm25.isna().sum())


In [ ]:
from sklearn.linear_model import LinearRegression

# Definiere die unabhängigen Variablen (Wetterdaten)
X = df_atlanta_pm25[['Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']]

# Definiere die abhängige Variable (PM2.5)
y = df_atlanta_pm25['Pm25']

# Lineares Regressionsmodell erstellen und trainieren
model = LinearRegression()
model.fit(X, y)

# Ergebnisse anzeigen
print('Koeffizienten:', model.coef_)
print('Intercept:', model.intercept_)


In [ ]:
# Berechnung des Bestimmtheitsmaßes (R²)
r_squared = model.score(X, y)

# Ausgabe des R²-Werts
print(f'Bestimmtheitsmaß (R²): {r_squared:.4f}')


Der R²-Wert ist recht niedrig. Das wundert mich nicht, da wir auch die Wettervariablen einbezogen haben, die kaum eine (lineare) Korrelation mit PM2.5 aufweisen. Wir wiederholen jeztt mal die Regressionsanalyse ohne die Variable "Humidity" und schauen, ob das den R²-Wert wverbessert. Wenn ja, würde das heißen, dass Humidity störendes Rauschen verursacht.

In [ ]:
# Entferne 'Humidity' aus den unabhängigen Variablen
X_new = df_atlanta_pm25[['Tavg', 'Prcp', 'Wspd', 'Pres']]

# Neues Modell ohne 'Humidity' trainieren
model_new = LinearRegression()
model_new.fit(X_new, y)

# Berechnung des neuen Bestimmtheitsmaßes (R²)
r_squared_new = model_new.score(X_new, y)

# Ausgabe des neuen R²-Werts
print(f'Neues Bestimmtheitsmaß (R²) ohne Luftfeuchtigkeit: {r_squared_new:.4f}')
